In [1]:
from tqdm import tqdm

tqdm.pandas()

# Loading Dataset

In [2]:
from datasets import load_dataset

dataset = load_dataset("go_emotions", "raw")
dataset.set_format(type="pandas")
df_train = dataset["train"][:]

/home/ossamaak0/miniconda3/envs/main/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
emotions = list(df_train.columns)[9:]
columns = ["text"] + emotions
# select only texts and emotions
df_train = df_train[columns]

In [4]:
# create new column with list of emotions (in strings)
df_train["ds_emotions"] = df_train[emotions].progress_apply(
    lambda x: x.index[x == 1].tolist(), axis=1
)

  0%|          | 0/211225 [00:00<?, ?it/s]

100%|██████████| 211225/211225 [00:12<00:00, 16675.19it/s]


In [32]:
# check if neutral can come with other emotions
values = df_train["ds_emotions"].value_counts().keys()
for value in values:
    if "neutral" in value:
        print(value)

['neutral']


In [89]:
# seed = 7102023
# # get random 20 examples
# samples = df_train.sample(20, random_state=seed)
# # drop emotions columns
# samples = samples.drop(emotions, axis=1)
# # head
# samples.head(5)
samples = df_train[df_train["ds_emotions"].apply(lambda x: "neutral" in x)]
samples = samples.sample(20, random_state=7102023)
# drop emotions columns
samples = samples.drop(emotions, axis=1)

# LLM Labeling

In [36]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

SYSTEM_MESSAGE = "You're an AI expert trained to analyze and categorize emotions present in text. Your goal is to analyze each piece of text according to instructions"

PROMPT_TEMPLATE = f"Given this comment {{comment}} , make an analysis of the emotions present in the comment according to this list of emotions ONLY {emotions[:-1]} or neutral if there's no emotion and after the analysis write the class/es that apply according to the given list (one, two or three classes) inside <answer> (classes) </answer> containing the classes."


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_MESSAGE),
        # few_shot_prompt,
        ("human", PROMPT_TEMPLATE),
    ]
)

In [60]:
groq_api = ""
openai_api = ""
fireworks_api = ""

In [117]:
# Define models
chat1 = ChatOpenAI(
    temperature=0,
    model_name="llama3:8b",
    openai_api_base="https://a6e4-34-31-3-189.ngrok-free.app/v1",
    openai_api_key=groq_api,
)

chain1 = final_prompt | chat1 | StrOutputParser()

chat2 = ChatOpenAI(api_key=openai_api)

chain2 = final_prompt | chat2 | StrOutputParser()

In [115]:
from out_parser import extract_emotions

In [119]:
samples["ollama8"] = None
# for index in range(len(samples)):
for index in tqdm(samples.index):
    comment = samples.loc[index]["text"]
    label = samples.loc[index]["ds_emotions"]
    response1 = chain1.invoke({"comment": comment})
    extracted_emotions = extract_emotions(response1)
    print(extracted_emotions)
    samples.at[index, "ollama8"] = extracted_emotions

  5%|▌         | 1/20 [00:07<02:27,  7.78s/it]

['neutral']


 10%|█         | 2/20 [00:11<01:38,  5.48s/it]

['amusement']


 15%|█▌        | 3/20 [00:14<01:11,  4.22s/it]

...

['anger', 'disgust']


 95%|█████████▌| 19/20 [00:59<00:02,  2.49s/it]

['amusement']


100%|██████████| 20/20 [01:02<00:00,  3.12s/it]

['excitement']


In [120]:
# save to csv
samples.to_csv("samples.csv", index=False)